# SciKera Classification

`pip install scikeras[tensorflow]`


In [8]:
import tensorflow as tf
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
from scikeras.wrappers import KerasClassifier

In [11]:
# Read data
iris = datasets.load_iris()

# Create a DataFrame
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df["target"] = iris.target

# Shuffle the data
df = df.sample(frac=1, random_state=0)

# Extract X and y
X = df.iloc[:, 0:4].values
y = df["target"].values

# Split data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=0, stratify=y
)
# Standardization
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [ ]:
def get_clf_model(hidden_layer_sizes, meta, compile_kwargs):

    model = tf.keras.Sequential()
    input_layer = tf.keras.layers.Input(shape=(meta["n_features_in_"]))
    model.add(input_layer)

    for i, hidden_layer_size in enumerate(hidden_layer_sizes):
        hidden_layer = tf.keras.layers.Dense(
            units=hidden_layer_size, activation="relu", name=f"hidden{i+1}"
        )
        model.add(hidden_layer)

    if meta["target_type_"] == "binary":
        n_output_units = 1
        output_activation = "sigmoid"
        loss = "binary_crossentropy"
    elif meta["target_type_"] == "multiclass":
        n_output_units = meta["n_classes_"]
        output_activation = "softmax"
        loss = "sparse_categorical_crossentropy"
    else:
        raise NotImplementedError(f"Unsupported task type: {meta['target_type_']}")

    output_layer = tf.keras.layers.Dense(n_output_units, activation=output_activation)
    model.add(output_layer)

    model.compile(
        optimizer=compile_kwargs["optimizer"],
        loss=loss,
        metrics=["accuracy"],
    )

    return model

In [ ]:
clf = KerasClassifier(
    model=get_clf_model,
    hidden_layer_sizes=(16, 3),
    optimizer="adam",
    optimizer__learning_rate=0.001,
    batch_size=16,
    epochs=200,
    validation_split=0.2,
    verbose=1,
)

In [12]:
clf.fit(X_train_std, y_train)

Epoch 1/200
6/6 [==============================] - 1s 32ms/step - loss: 1.0746 - accuracy: 0.2857 - val_loss: 1.0005 - val_accuracy: 0.6667
Epoch 2/200
6/6 [==============================] - 0s 6ms/step - loss: 1.0331 - accuracy: 0.4286 - val_loss: 0.9728 - val_accuracy: 0.7143
Epoch 3/200
6/6 [==============================] - 0s 6ms/step - loss: 0.9933 - accuracy: 0.5000 - val_loss: 0.9514 - val_accuracy: 0.7143
Epoch 4/200
6/6 [==============================] - 0s 6ms/step - loss: 0.9615 - accuracy: 0.5714 - val_loss: 0.9307 - val_accuracy: 0.7143
Epoch 5/200
6/6 [==============================] - 0s 6ms/step - loss: 0.9317 - accuracy: 0.6310 - val_loss: 0.9119 - val_accuracy: 0.7143
Epoch 6/200
6/6 [==============================] - 0s 6ms/step - loss: 0.9047 - accuracy: 0.6905 - val_loss: 0.8934 - val_accuracy: 0.7143
Epoch 7/200
6/6 [==============================] - 0s 6ms/step - loss: 0.8804 - accuracy: 0.6905 - val_loss: 0.8764 - val_accuracy: 0.7143
Epoch 8/200
6/6 [=========

KerasClassifier(
	model=<function get_clf_model at 0x00000179B3E8F9C0>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=adam
	loss=None
	metrics=None
	batch_size=16
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.2
	shuffle=True
	run_eagerly=False
	epochs=200
	hidden_layer_sizes=(16, 3)
	optimizer__learning_rate=0.001
	class_weight=None
)

In [13]:
clf.score(X_test_std, y_test)

3/3 [==============================] - 0s 1ms/step


0.9555555555555556